# Serie Trigonométrica de Fourier
---

## **Como utilizar el código:**
La función principal es: st_fourier(f, p). Esta nos dará la expresión de la **STF** correspondiente a $f(t)$ acompañada por un gráfico.

* **f :** es expresión de la $f(t)$ a la cual se le quiere calcular la **STF**,
 resulta ser una cadena cuya estructura es la siguiente:
    ```python
"[exprsion_1,intervalo_1][exprsion_2,intervalo_2] ... [expresion_n,intervalo_n]"
    ```
* **p :** representa el período de $f(t)$, debe tener un valor igual a la distancia entre la cota inferior y la cota superior de $D_f$. Será un número representado mediante una cadena.

## **Como escribir la función de entrada:**

### Expresiones:
**expresion_n:** es la expresión propiamente dicha que toma $f(t)$ en el intervalo_n, la misma se debe escribir tal como si la escribiéramos en python. Ejemplo: $\ \frac{t^2}{t\ +\ 2}\ ⇒\ $ expresion_n = ( t ** 2 ) / ( t + 2 )

### Intervalos:
**intervalo_n:** representa para que valores de $t$ está definida expresion_n. Aquí van las diferentes formas de escribirlo:

* $a\ ∧\ b:\$ (condición_1) \& ( condición_2 ) \& ... ( condición_n )

* $a\ ∨\ b:\$ (condición_1) | ( condición_2 ) |  ... ( condición_n )

* $|\ t\ |\ ≤\ a :\ $ Abs(t) <= a

## **Los números $\ \pi\ $ y $\ e$**
Para poder utilizar estas números, se los debe escribir de las siguiente manera:

**$\pi:$** pi

**$e:$** E

### Ejemplos:
Para escribir $sen(2 \pi t):$
    
```python
sin(2*pi*t)
```
Para escribir $e^{t}:$

```python
E**t
```

## Ejemplo Final

En base a lo anterior, si queremos obtener la **STF** de una función como:

${\displaystyle
f(t) =
\begin{cases}
    \large -\frac{\pi}{4} & \text{si } \ t \ ∈ \ (-\pi,0) \\
    \large \frac{\pi}{4}  & \text{si } \ t \ ∈ \ (0,\pi) \\
\end{cases}
}$

Nuestros parámetros f y p serían:
```python
f = "[-pi/4,(-pi<t)&(t<0)][pi/4,(0<t)&(t<pi)]"
p = "2*pi"
```




In [ ]:
import sympy as sp
from sympy import Union, parse_expr
import plotly.graph_objects as go
import numpy as np
import re

t = sp.Symbol("t")
n, i = sp.symbols("n,i", integer=True, positive=True) # "n" e "i" son naturales

def st_fourier(funcion: str, periodo: str):
    f = transformar_expresion(funcion)
    T = parse_expr(periodo)
    w = (2 * sp.pi) / T
    L = T / 2

    a0 = coeficiente(f, L)
    an = coeficiente(sp.piecewise_fold(f * sp.cos(n * w * t)), L)
    bn = coeficiente(sp.piecewise_fold(f * sp.sin(n * w * t)), L)

    an_cos = an * sp.cos(n * w * t)
    bn_sin = bn * sp.sin(n * w * t)

    stf = a0 / 2 + sp.Sum(an_cos + bn_sin, (n, 1, i))

    graficar_serie(a0, stf, f)

def coeficiente(funcion, L):
    f_partes = funcion.as_expr_set_pairs()
    integral = 0

    for exp, itv in f_partes:
        if isinstance(itv, Union):
            # si el intervalo es una unión, hay que iterarlo
            integral += sum(sp.integrate(exp, (t, arg)) for arg in itv.args)
        else:
            integral += sp.integrate(exp, (t, itv))

    return (1 / L) * integral

# devuelve la espresión simbólica de la función elegida
def transformar_expresion(funcion: str):
    # separo cada parte de la función en una lista
    resultados = re.findall(r"\[[^\[\]]+\]", funcion)

    f_partes = [re.sub(r"\[|\]", "", s) for s in resultados]
    f_partes = [parte.split(",") for parte in f_partes]

    funcion_s = [(parse_expr(exp), parse_expr(cond)) for exp, cond in f_partes]

    return sp.Piecewise(*funcion_s)

def crear_deslizador(fig):
    steps = []

    for i in range(0, len(fig.data) - 2):
        step = dict(
            method="update",
            args=[{"visible": [False] * (len(fig.data) - 2) + [True] * 2}], # los gráficos de f y a0/2 quedan estáticos
            label=f"n= {i + 1}",
        )
        step["args"][0]["visible"][i] = True # visualiza el gráfico para el "n" seleccionado
        steps.append(step)

    # visualizar el gráfico de f y la serie para los primeros 11 armónicos
    fig.data[-1].visible = True
    fig.data[-2].visible = True
    fig.data[10].visible = True

    return [
        dict(
            active=10,
            currentvalue={"prefix": "Sumas Parciales: "},
            steps=steps,
        )
    ]

def agregar_grafico(graficos, func, t_vals, mode, name):
    # optimiza la evaluación de la función
    lamb_f = sp.lambdify(t, func, "numpy")

    graficos.append(go.Scatter(
        x=t_vals,
        y=lamb_f(t_vals),
        line=dict(dash=mode),
        visible=False,
        name=name,
    ))

def crear_graficos(a0, stf, funcion):
    t_vals = np.linspace(-10, 10, 3000)
    graficos = []

    for n_max in range(1, 31):
        agregar_grafico(graficos, stf.subs(i,n_max), t_vals,
            "solid", "$\large Sf(t)$"
        )

    agregar_grafico(graficos, funcion, t_vals,
        "solid", "$\large f(t)$"
    )
    agregar_grafico(graficos, [a0/2,a0/2], [-15,15],
        "dash", "$\large a_0/2$"
    )

    return graficos

def graficar_serie(a0, stf, funcion):
    graficos = crear_graficos(a0, stf, funcion)
    fig = go.Figure(data=graficos)
    stf = stf.subs(i,sp.oo)

    fig.update_layout(
        title=dict(text=f"$\Large Sf(t) = {sp.latex(stf)}$", x=0.5),
        width=1300,
        height=800,
        sliders=crear_deslizador(fig),
    )
    fig.update_xaxes(range=[-5, 5])

    fig.show()

# **Ahora un par de ejemplos...**

### **Caso 1: Funciones Impares**

Sabemos que cuando tenemos un producto de funciones impares el resultado es una función par. En cambio, cuando una de ellas es par, el resultado es una función impar.
Siguiendo estas afirmaciones y aprovechando las propiedades de la paridad de las funciones sobre las integrales, cuando la función $f(t)$ a la que le queremos calcular la **STF** es impar, los coeficientes resultaran de la siguiente manera:

${\displaystyle a_0 = \frac{1}{L}\int_{-L}^{L} f(t)\ dt\ =\ 0}$

$ {\displaystyle a_n =\frac{1}{L}\int_{-L}^{L} f(t) · cos(n\ ω\ t)\ dt\ =\ 0}$

$ {\displaystyle b_n = \frac{1}{L}\int_{-L}^{L} f(t) · sen(n\ ω\ t)\ dt\ = \frac{2}{L}\int_{0}^{L} f(t)\ ·\ sen(n\ ω\ t)\ dt\ \not=\ 0}$

En base a estos resutados, podemos concluir que la **STF** de una función impar estará constituida únicamente por **senos**.

\
Para visualizar esto, tomemos como ejemplo:

$ { \displaystyle f(t) =
  \begin{cases}
    e^{t}, &  \text{si } t\ ∈\ (0,1)\quad ∧ \quad f(t) = f(t+2) \\
    -e^{-t}, & \text{si } t\ ∈\ (-1,0)
  \end{cases}
}$

In [ ]:
f = "[E**t,(0<t)&(t<1)][-E**(-t),(-1<t)&(t<0)]"
p = "2"
st_fourier(f,p)

### **Caso 2: Funciones Pares**

Si tomamos las ideas del caso anterior, podemos deducir que:

$ {\displaystyle a_0 = \frac{1}{L}\int_{-L}^{L} f(t)\ dt\ }$

$ {\displaystyle a_n = \frac{1}{L}\int_{-L}^{L} f(t) · cos(n\ ω\ t)\ dt\ \not=\ 0}$

$ {\displaystyle b_n = \frac{1}{L}\int_{-L}^{L} f(t) · sen(n\ ω\ t)\ dt\ =\ 0}$

Por lo tanto, la **STF**  quedará constituida por **cosenos**.

\
Como ejemplo tomaremos:

${\displaystyle f(t)\ =\ sen(t),\ \ si\ t\ ∈\ (0,π)\quad ∧ \quad f(t) = f(t+π)}$

In [ ]:
f = "[sin(t),(0<t)&(t<pi)]"
p = "pi"
st_fourier(f,p)

### **Caso 3: Funciones Periódicamente Alternadas**

La condición que debe cumplir una función $f(t)$ para que sea periódicamente alternada es:

${\displaystyle f(t) = -f(t + \frac{T}{2})}$

Si $f(t)$ cumple esta condición, resulta que los coeficientes $a_{2k}$ y $b_{2k}$ serán nulos.

\
Como ejemplo, tomemos:

$ { \displaystyle f(t) =
  \begin{cases}
    5, &  \text{si } t\ ∈\ (0,2)\quad ∧ \quad f(t) = f(t+4) \\
    1, & \text{si } t\ ∈\ (2,4)
  \end{cases}
}$

\
**Nota:** el hecho de que el $a_0$ sea nulo o no, depende del desplazamiento de la función. En caso de que el mismo sea cero, entonces el $a_0$ también se anulará.

In [ ]:
f = "[5,(0<t)&(t<2)][1,(2<=t)&(t<4)]"
p = "4"
st_fourier(f,p)

### **Caso 4: Funciones Sin Paridad**

Puede ocurrir que una función no cumpla ni la condición de una función par o de una impar ¿qué ocurriría en estos casos? Pues, al no poseer paridad (en principio) no se anularía ningún coeficiente. En conclusión, tendríamos una **STF** constituida tanto por **cosenos** como por **senos**.

\
Un ejemplo de este tipo de funciones puede ser:

${\displaystyle f(t) = t^2, \ \ si\ t\ ∈\ (0,2π) \quad ∧ \quad f(t) = f(t + 2π)}$

In [ ]:
f = "[t**2,(-0<t)&(t<2*pi)]"
p = "2*pi"
st_fourier(f,p)

### **Caso 5: Funciones Impares Desplazadas**

Puede suceder que, debido a un desplazamiento vertical, una función $f(t)$ parezca no presentar paridad alguna. En estos casos, podemos definir otra función $g(t)$ tal que:

$ g(t)\ =\ f(t)\ -\ \frac{a_0}{2} \quad$  ($\ \frac{a_0}{2}\ $ correspondiente a $f(t)\ $)

De está manera, $g(t)$ resulta ser una función impar y podemos aprovechar la propiedades que trae con ella para calcular los coeficientes.

Luego, debemos volver a la función original, por lo tanto, la relación entre la **STF** de $f(t)$ y la de $g(t)$ será:

$Sf(t)\ =\ Sg(t)\ +\ \frac{a_0}{2}$

\
Referenciando este caso, tomemos la función del **caso 1** pero con una pequeña modificación:

${\displaystyle f(t)\ =\ t\ +\ 2,\ \ si\ \ t\ ∈\ (-1,1)\quad ∧ \quad f(t) = f(t+2)}$

In [ ]:
f = "[t + 2,(-1<t)&(t<1)]"
p = "2"
st_fourier(f,p)

### **Caso 6: Una Combinación...**

Puede ocurrir que una función sea par (o impar) y presente simetría de media onda. En estas circunstancias, la **STF** no solo estará compuesta o bien por cosenos o bien por senos, dependiendo de la paridad de la función, sino que también los coeficientes pares se anularán debido a la simetría de media onda.

\
Para visualizar estos casos, tomemos como ejemplo una función par y con simetría de media onda:

${\displaystyle f(t) = |\ t\ |\ -\ 1, \ \ si \ \ t\ ∈\ (-2,2) \quad ∧ \quad f(t) = f(t + 4)}$

In [ ]:
f = "[Abs(t)-1,(-2<t)&(t<2)]"
p = "4"
st_fourier(f,p)

## **Ejemplo con Módulos en el Intervalo**


$ {\displaystyle f(t) =
  \begin{cases}
    1, &  \text{si }\ 1 \le |\ t\ | \le\ 2 \quad ∧ \quad f(t) = f(t+4) \\
    t, & \text{si }\ 0 \lt\ t\ \lt\ 1  \\
    -t, & \text{si }\ -1 \lt\ t\ \lt\ 0
  \end{cases}
}$

In [ ]:
f = "[1,(1<Abs(t))&(Abs(t)<2)][t,(0<t)&(t<1)][-t,(-1<t)&(t<0)]"
p = "4"
st_fourier(f,p)

## **Ejemplo en Clase**


${\displaystyle f(t) = sen(3t), \ \ si \ \ t\ ∈\ (-0, \frac{2}{3}\pi) \quad ∧ \quad f(t) = f(t + \frac{2}{3}\pi)}$

In [ ]:
f = "[sin(3*t),(0<t)&(t<(2*pi)/3)]"
p = "(2*pi)/3"
st_fourier(f,p)